# Recommender

With the model created, we want to be able to take the url of two decks and see which one is better

In [1]:
import pandas as pd
import numpy as np
from sklearn import linear_model
import pickle
from load import load_single_deck
from load import decode_single_deck

Using pickle we can load the model

In [27]:
filename = 'finalized_linear_model.sav'
lm = pickle.load(open(filename, 'rb'))

These are the two decks we're going to look at, these are in the format the use would get from the website.

In [3]:
deck1 = "https://www.keyforgegame.com/deck-details/f62c8aec-ab9f-4623-947d-8cfa412a0290"
deck2 = "https://www.keyforgegame.com/deck-details/8af0f140-b4a6-4490-a367-2cc1aa0d6b10"

Next we need to convert this into a format that the model can use.

In [4]:
deck1_code = deck1[-36:]
deck2_code = deck2[-36:]

In [5]:
deck1_data = load_single_deck(deck1_code)
deck2_data = load_single_deck(deck2_code)

In [6]:
deck1_df = decode_single_deck(deck1_data)
deck2_df = decode_single_deck(deck2_data)
deck_df = pd.concat([deck1_df, deck2_df], axis=0)
deck_df

,deck_id,deck_name,deck_wins,deck_losses,deck_expansion,deck_list,houses
0,f62c8aec-ab9f-4623-947d-8cfa412a0290,"Gunhild, Orange Saloon Sensei",0,0,341,"[b361d7e2-6873-4890-8f87-702d9c89c5ad, b361d7e...","[Brobnar, Mars, Shadows]"
0,8af0f140-b4a6-4490-a367-2cc1aa0d6b10,Arbitrarily Iron Thjodolf,0,0,341,"[a0f1146a-6df3-4568-8fb1-d6845615d833, 1ad7152...","[Brobnar, Mars, Untamed]"


In [7]:
card_df = pd.read_csv("data/cards.csv")

In [9]:
if 'Brobnar' in deck_df['houses'].values:
    deck_df['house_brobnar'] = True
else:
    deck_df['house_brobnar'] = False
if 'Dis' in deck_df['houses'].values:
    deck_df['house_dis'] = True
else:
    deck_df['house_dis'] = False
if 'Sanctum' in deck_df['houses'].values:
    deck_df['house_sanctum'] = True
else:
    deck_df['house_sanctum'] = False
if 'Mars' in deck_df['houses'].values:
    deck_df['house_mars'] = True
else:
    deck_df['house_mars'] = False
if 'Untamed' in deck_df['houses'].values:
    deck_df['house_untamed'] = True
else:
    deck_df['house_untamed'] = False
if 'Shadows' in deck_df['houses'].values:
    deck_df['house_shadows'] = True
else:
    deck_df['house_shadows'] = False
if 'Logos' in deck_df['houses'].values:
    deck_df['house_logos'] = True
else:
    deck_df['house_logos'] = False

In [10]:
deck_df = deck_df.drop("houses", axis=1)
deck_df.drop(deck_df.columns[deck_df.columns.str.contains('Unnamed',case = False)],axis = 1, inplace = True)

In [11]:
deck_df.head()

,deck_id,deck_name,deck_wins,deck_losses,deck_expansion,deck_list,house_brobnar,house_dis,house_sanctum,house_mars,house_untamed,house_shadows,house_logos
0,f62c8aec-ab9f-4623-947d-8cfa412a0290,"Gunhild, Orange Saloon Sensei",0,0,341,"[b361d7e2-6873-4890-8f87-702d9c89c5ad, b361d7e...",False,False,False,False,False,False,False
0,8af0f140-b4a6-4490-a367-2cc1aa0d6b10,Arbitrarily Iron Thjodolf,0,0,341,"[a0f1146a-6df3-4568-8fb1-d6845615d833, 1ad7152...",False,False,False,False,False,False,False


In [12]:
def change_card_array_ids_to_names(card_list):
    new_card_list = []
    for i in card_list:
        card = card_df.loc[(card_df['card_id'] == i)]['card_title'].item()
        new_card_list.append(card)
    return(new_card_list)

In [13]:
series_of_cards = card_df['card_title']
list_of_cards = list(dict.fromkeys(series_of_cards))
list_of_cards.sort()
list_of_cards[568] = "REDACTED"
list_of_cards

['1-2 Punch',
 'A Fair Game',
 'Abond the Armorsmith',
 'Agent Hoo-man',
 'Ammonia Clouds',
 'Anahita the Trader',
 'Ancient Bear',
 'Ancient Yurk',
 'Anger',
 'Angwish',
 'Annihilation Ritual ',
 'Anomaly Exploiter',
 'Archimedes',
 'Arise!',
 'Armageddon Cloak',
 'Aubade the Grim',
 'Autocannon',
 'Backup Copy',
 'Bad Penny',
 'Bait and Switch',
 'Banish',
 'Banner of Battle',
 'Barehanded',
 'Baron Mengevin',
 'Barrister Joya',
 'Batdrone',
 'Battle Fleet',
 'Bear Flute',
 'Begone!',
 'Bellowing Patrizate',
 'Bigtwig',
 'Bilgum Avalanche',
 'Binate Rupture',
 'Binding Irons',
 'Bingle Bangbang',
 'Biomatrix Backup',
 'Blinding Light',
 'Blood Money',
 'Blood of Titans',
 'Bloodshard Imp',
 'Blypyp',
 'Bonerot Venom',
 'Booby Trap',
 'Bordan the Redeemed',
 'Bouncing Deathquark',
 'Brain Eater',
 'Brain Stem Antenna',
 'Brammo',
 'Brend the Fanatic',
 'Briar Grubbling',
 'Brobnar Ambassador',
 'Bronze Key Imp',
 'Brothers in Battle',
 'Bulleteye',
 'Bulwark',
 'Bumblebird',
 'Bumpsy'

In [14]:
deck_df['card_name_list'] = deck_df['deck_list'].apply(change_card_array_ids_to_names)

In [15]:
deck_df.head()

,deck_id,deck_name,deck_wins,deck_losses,deck_expansion,deck_list,house_brobnar,house_dis,house_sanctum,house_mars,house_untamed,house_shadows,house_logos,card_name_list
0,f62c8aec-ab9f-4623-947d-8cfa412a0290,"Gunhild, Orange Saloon Sensei",0,0,341,"[b361d7e2-6873-4890-8f87-702d9c89c5ad, b361d7e...",False,False,False,False,False,False,False,"[Anger, Anger, Follow the Leader, Punch, Ungua..."
0,8af0f140-b4a6-4490-a367-2cc1aa0d6b10,Arbitrarily Iron Thjodolf,0,0,341,"[a0f1146a-6df3-4568-8fb1-d6845615d833, 1ad7152...",False,False,False,False,False,False,False,"[Follow the Leader, Loot the Bodies, Tremor, U..."


In [16]:
for n in list_of_cards:
    deck_df[n] = deck_df['card_name_list'].astype(str).str.count(n)

In [17]:
deck_df.loc[deck_df.deck_expansion == 435, 'expansion_coa'] = True #'Call of the Archons'
deck_df.loc[deck_df.deck_expansion == 341, 'expansion_coa'] = False
deck_df.loc[deck_df.deck_expansion == 435, 'expansion_aoa'] = True #'Call of the Archons'
deck_df.loc[deck_df.deck_expansion == 341, 'expansion_aoa'] = False

In [18]:
deck_df.head()

,deck_id,deck_name,deck_wins,deck_losses,deck_expansion,deck_list,house_brobnar,house_dis,house_sanctum,house_mars,...,Zysysyx Shockworm,Zyzzix the Many,REDACTED,Æmber Conduction Unit,Æmber Imp,Æmberspine Mongrel,“John Smyth”,“Lion” Bautrem,expansion_coa,expansion_aoa
0,f62c8aec-ab9f-4623-947d-8cfa412a0290,"Gunhild, Orange Saloon Sensei",0,0,341,"[b361d7e2-6873-4890-8f87-702d9c89c5ad, b361d7e...",False,False,False,False,...,0,2,0,0,0,0,0,0,False,False
0,8af0f140-b4a6-4490-a367-2cc1aa0d6b10,Arbitrarily Iron Thjodolf,0,0,341,"[a0f1146a-6df3-4568-8fb1-d6845615d833, 1ad7152...",False,False,False,False,...,0,0,0,0,0,0,0,0,False,False


In [19]:
deck_df = deck_df.drop("deck_list", axis=1)
deck_df = deck_df.drop("card_name_list", axis=1)

In [29]:
X = deck_df
X = X.drop("deck_id", axis=1)
X = X.drop("deck_name", axis=1)
X = X.drop("deck_wins", axis=1)
X = X.drop("deck_losses", axis=1)
X = X.drop("house_shadows", axis=1)
X = X.drop("house_mars", axis=1)
X = X.drop("house_untamed", axis=1)
X = X.drop("house_logos", axis=1)
X = X.drop("house_dis", axis=1)
X = X.drop("house_sanctum", axis=1)
X = X.drop("house_brobnar", axis=1)
X = X.drop("deck_expansion", axis=1)
X = X.drop("expansion_coa", axis=1)
X = X.drop("expansion_aoa", axis=1)

In [30]:
X

,1-2 Punch,A Fair Game,Abond the Armorsmith,Agent Hoo-man,Ammonia Clouds,Anahita the Trader,Ancient Bear,Ancient Yurk,Anger,Angwish,...,Z.Y.X. Researcher,Zorg,Zysysyx Shockworm,Zyzzix the Many,REDACTED,Æmber Conduction Unit,Æmber Imp,Æmberspine Mongrel,“John Smyth”,“Lion” Bautrem
0,0,0,0,0,0,0,0,0,2,0,...,0,0,0,2,0,0,0,0,0,0
0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [31]:
lm

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [32]:
predictions = lm.predict(X)
print(predictions)

[[0.8223536 ]
 [0.01537275]]


In [34]:
deck_df['deck_name']

0    Gunhild, Orange Saloon Sensei
0        Arbitrarily Iron Thjodolf
Name: deck_name, dtype: object